In [1]:
#All these packages need to be installed from pip
#For ML
import sklearn
import sklearn.feature_extraction.text
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups, make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer  #Feature extraction
from sklearn.naive_bayes import MultinomialNB #Our learner.
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, SelectFromModel

import pandas as pd
import nltk #For tokenizing and normalizing
import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
from matplotlib.colors import ListedColormap
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

%matplotlib inline

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import requests
import json
import math

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pandas.read_pickle('cmv_full_features.pkl')
df[:3]

,com_text,com_delta_received,com_delta_from_op,com_upvotes,sub_id,sentences,com_avg_pt_depth,sub_text,tokenized_sub,normalized_sub,tokenized_com,normalized_com,kmeans,kmeans_inter,KL,JS
0,I think a lot depends on the process by which ...,True,True,4967,4k83qf,"[[I, think, a, lot, depends, on, the, process,...",13.769231,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...","[I, think, a, lot, depends, on, the, process, ...","[think, lot, depend, process, women, distribut...",4,power word words use understand earth using wo...,0.357308,0.083299
1,What would be the alternative? An anarchosynd...,False,False,952,4k83qf,"[[What, would, be, the, alternative, ?], [An, ...",11.000000,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...","[What, would, be, the, alternative, ?, An, ana...","[would, altern, anarchosyndicalist, commune, s...",4,power word words use understand earth using wo...,0.000000,0.000000
2,"Empirically speaking, the only country that tr...",False,False,230,4k83qf,"[[Empirically, speaking, ,, the, only, country...",10.500000,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...","[Empirically, speaking, ,, the, only, country,...","[empir, speak, onli, countri, tri, work, prett...",4,power word words use understand earth using wo...,0.000000,0.000000


In [4]:
F_DF = pd.read_pickle('extracted_features.pkl')
F_DF

,all_features_f,pvalues_f,scores_f,all_features_m,scores_m,all_features_t1,scores_t1,all_features_t2,scores_t2
0,com_upvotes,0.000000e+00,2598.615876,com_upvotes,0.029531,com_upvotes,0.400377,com_upvotes,0.019197
1,large group,1.635654e-13,54.456131,things,0.004259,sold,0.025940,com_avg_pt_depth,0.002823
2,JS,2.971789e-13,53.280571,just,0.004141,additional,0.019782,KL,0.002605
3,KL,3.674229e-13,52.862969,does,0.003941,theoretically,0.018814,JS,0.002588
4,outrage,4.733387e-13,52.364524,point,0.003561,extra,0.017462,kmeans,0.002266
5,mexican,8.204640e-13,51.282426,com_avg_pt_depth,0.003424,difficult,0.017413,think,0.001842
6,outcomes,1.344589e-10,41.275076,need,0.003190,giving,0.017193,people,0.001700
7,audiences,2.990624e-10,39.710802,pretty,0.003175,annoying,0.016848,just,0.001634
8,airline,5.830431e-10,38.405535,make,0.003133,airlines,0.016255,don,0.001416
9,accountable,1.000888e-09,37.349810,time,0.003096,animal,0.016221,like,0.001358


In [11]:
f_features = F_DF.iloc[:, 0].tolist()
m_features = F_DF.iloc[:, 3].tolist()
t_features = F_DF.iloc[:, 7].tolist()

In [29]:
f_in3 = set(f_features)&set(m_features)&set(t_features)
f_in3 = list(f_in3)
f_in3

['com_avg_pt_depth', 'com_upvotes']

In [30]:
f_in2 = list(set(f_features)&set(m_features))+ list(set(m_features)&set(t_features)) + list(set(f_features)&set(t_features))
f_in2 = list(set(f_in2))
f_in2

['KL',
 'com_upvotes',
 'make',
 'don',
 'just',
 'point',
 'lot',
 'does',
 'com_avg_pt_depth',
 'JS']

In [31]:
words_in2 = f_in2[2:-2]
words_in2

['make', 'don', 'just', 'point', 'lot', 'does']

In [58]:
d = {}
type(d)
d['2ord'] = 8
d

{'2ord': 8}

In [62]:
d = {}
for word in words_in2:
    d[word] = df["com_text"].apply(lambda x: word in x).tolist()

wordsDF = pd.DataFrame(d)
wordsDF
#df[df['com_delta_received']['make' in df["com_text"]]]

,does,don,just,lot,make,point
0,False,True,False,True,True,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,True,False,False,False
5,False,False,False,False,True,False
6,False,False,False,True,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [48]:
df[df["com_text"].apply(lambda x: 1-('make' in x))]['com_delta_received'].mean()

com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
com_delta_received    0.021741
                        ...   
com_delta_received    0.021741
com_delt

In [63]:
cmv_df = pd.read_pickle('cmv_full_features.pkl')
words_in2 = ['make', 'don', 'just', 'point', 'lot', 'does']
for word in words_in2:
    cmv_df[word] = cmv_df["com_text"].apply(lambda x: word in x)

In [64]:
cmv_df[:3]

,com_text,com_delta_received,com_delta_from_op,com_upvotes,sub_id,sentences,com_avg_pt_depth,sub_text,tokenized_sub,normalized_sub,...,kmeans,kmeans_inter,KL,JS,make,don,just,point,lot,does
0,I think a lot depends on the process by which ...,True,True,4967,4k83qf,"[[I, think, a, lot, depends, on, the, process,...",13.769231,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...",...,4,power word words use understand earth using wo...,0.357308,0.083299,True,True,False,False,True,False
1,What would be the alternative? An anarchosynd...,False,False,952,4k83qf,"[[What, would, be, the, alternative, ?], [An, ...",11.000000,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...",...,4,power word words use understand earth using wo...,0.000000,0.000000,False,False,False,False,False,False
2,"Empirically speaking, the only country that tr...",False,False,230,4k83qf,"[[Empirically, speaking, ,, the, only, country...",10.500000,Supreme executive power derives from a mandate...,"[Supreme, executive, power, derives, from, a, ...","[suprem, execut, power, deriv, mandat, mass, f...",...,4,power word words use understand earth using wo...,0.000000,0.000000,False,False,False,False,False,False
